In [3]:
import numpy as np

In [6]:
### Solutions to the Camassa--Holm equation
# i.e. functions to generate feature/label pairs

class _Solution:
    def __call__(self, point):
        """Evaluates the solution at the particular point."""
        raise NotImplementedError
        
    def on_grid(self, grid, extra=0):
        """Evaluates the solution on a grid of points.
        
        :[(float, float)] grid: The grid points on which to evaluate, as
            returned by either fine_grid or coarse_grid.
        :int extra: A nonnegative integer specifying that the array should be
            larger by this many entries. (To add extra data to later without the
            overheard of creating another array.)"""
        return np.array([self(grid_point) for grid_point in grid] 
                        + [0 for _ in range(extra)])
    
    def on_vals(self, tvals, xvals):
        """Evalutes the solution on a grid of points corresponding to
        the mesh grid produced by :tvals: and :xvals:
        """
        return np.array([[self((t, x)) for x in xvals] for t in tvals])

    
class Peakon(_Solution):
    """Simplest solution to the k=0 Camassa-Holm equation.
    
    Represents a soliton with a peak.
    """
    
    def __init__(self, c, **kwargs):
        """Peakons have precisely one parameter :c: defining their height and 
        speed. (We could also add an additional parameter defining their initial
        location, for consistency with TwoPeakon, but that's essentially
        unnecessary)
        """
        self.c = c
        super(Peakon, self).__init__(**kwargs)
        
    def __call__(self, point):
        t, x = point
        return self.c * np.exp(-1 * np.abs(x - self.c * t))
    

class TwoPeakon(_Solution):
    """Represents two solitons with peaks! Nonlinear effects start determining
    it location and magnitude.
    """
    
    def __init__(self, x1, x2, p1, p2, **kwargs):
        """TwoPeakons have essentially four parameters defining them.

        :x1: and :x2: are the initial locations of the peakons.
        :p1: and :p2: are the initial heights of the peakons. They must be
            positive. (Not zero!)
        """
        if x1 > x2:
            x1, x2 = x2, x1
            p1, p2 = p2, p1        
        
        X1 = np.exp(x1)
        X2 = np.exp(x2)
        a = p1 * p2 * (X1 - X2)
        b = (p1 + p2) * X2
        c = -X2
        discrim = np.sqrt(b ** 2 - 4 * a * c)
        twice_a = 2 * a
        
        l1 = (-b + discrim) / twice_a
        l2 = (-b - discrim) / twice_a
        
        a1 = (-1  + l2 * p2) * X2 / (p2 * (-l1 + l2))
        a2 = X2 - a1
        
        self.l1 = l1
        self.l2 = l2
        self.a1 = a1
        self.a2 = a2
        
        super(TwoPeakon, self).__init__(**kwargs)
        
    def __call__(self, point):
        t, x = point
        
        a1 = self.a1 * np.exp(t / self.l1)
        a2 = self.a2 * np.exp(t / self.l2)
        l1 = self.l1
        l2 = self.l2
        
        a1l1 = a1 * l1
        a2l2 = a2 * l2
        a1l1l1 = a1l1 * l1
        a2l2l2 = a2l2 * l2
        a1_a2 = a1 + a2
        tmp1 = a1l1l1 + a2l2l2
        tmp2 = a1l1 + a2l2
        
        # Can probably optimise this some more in terms of factorising out the
        # t dependence
        p1 = tmp1 / (l1 * l2 * tmp2)
        p2 = a1_a2 / tmp2
        x1 = np.log((a1l1l1 * a2 + a2l2l2 * a1 - 2 * a1l1 * a2l2) / tmp1)
        x2 = np.log(a1_a2)
        
        first_peakon = p1 * np.exp(-1 * np.abs(x - x1))
        second_peakon = p2 * np.exp(-1 * np.abs(x - x2))
        return first_peakon + second_peakon
    
# And for more peakons the algebra gets disgusting (and more importantly, slow), so 
# we'll leave it at two peakons for exact solutions.